In [1]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import dask

####################you will need to change some paths here!#####################
#list of input files
#adir_aviso = 'F:/data/project_data/NASA_biophysical/aviso/'
#filename_northpac_eddies=adir_aviso + 'eddy_trajectory_19930101_20170106_north_pacific_2020_10_06.nc'
#filename_cpr_eddy=adir_data + 'eddy_cpr_data_north_pacific_2020_10_06.nc'
#filename_aviso_all=adir_aviso+'/eddy_trajectory_19930101_20170106.nc'   #From AVISO  website
#filename_cpr_all=adir_data + '/All CPR Sample catalogue with eddy info_2020_10_06.nc'
#filename_cpr_all=adir_data+'All_CPR_Sample_catalogue_with_eddy_info_2020_10_06_origin_file.nc'

ctype = 'goa'  #'CPR'
if ctype=='goa':
    adir_data = 'f:/data/project_data/NASA_biophysical/collocated_data/'
    filename_origin_in=adir_data + 'NPPSD_GOA_allseabird_full_eddy_info.nc'
    filename_origin_out=adir_data + 'NPPSD_GOA_allseabird_full_eddy_info_envdata'
else:
    adir_data = 'F:/data/project_data/NASA_biophysical/collocated_data/CPR/'
    filename_origin_in=adir_data+'All_CPR_Sample_catalogue_with_eddy_info_2020_10_06_origin_file.nc'
    filename_origin_out=adir_data+'All_CPR_Sample_catalogue_with_eddy_info_2020_10_06_origin_file_with_data'

#################################################################################
#some of the data is on pangeo gcp, some on AWS
import sys
sys.path.append('./../cloud_science/subroutines/')  #where your
#from get_data_pangeo import get_data
from get_data_local import get_data_360


# Read in data and put in a dictionary

In [2]:
data,clim = get_data_360()

C:\Users\gentemann\Miniconda3\envs\satenv\lib\site-packages\xarray\core\indexing.py:1361: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


In [3]:
ds_bird = xr.open_dataset(filename_origin_in)
ds_bird['time'] = ds_bird.time64 if ctype=='goa' else print('CPR data')
ds_bird.close()
#ds_bird['lon'] = (ds)bird['lon'] + 180) % 360 - 180  #make -180 to 180
ds_bird_save = ds_bird.copy(deep=True)
ds_bird

<xarray.Dataset>
Dimensions:             (observation_number: 1830, track: 765)
Coordinates:
  * track               (track) int32 4438 4918 5213 ... 270829 270892 271234
  * observation_number  (observation_number) int32 0 1 2 3 ... 1827 1828 1829
Data variables:
    amplitude           (track, observation_number) float32 ...
    cyclonic_type       (track, observation_number) float64 ...
    lat                 (track, observation_number) float32 ...
    lon                 (track, observation_number) float32 ...
    speed_average       (track, observation_number) float32 ...
    speed_radius        (track, observation_number) float32 ...
    time64              (track, observation_number) datetime64[ns] ...
    time                (track, observation_number) datetime64[ns] ...
Attributes:
    Metadata_Conventions:      Unidata Dataset Discovery v1.0
    title:                     Mesoscale Eddies in Altimeter Observations of SLA
    institution:               CNES/CLS in collaboration with Oregon State Un...
    project:                   SSALTO/DUACS
    creator_url:               http://www.aviso.altimetry.fr
    creator_email:             aviso@altimetry.fr
    license:                   http://www.aviso.altimetry.fr/fileadmin/docume...
    summary:                   This dataset contains eddy atlas from two-sate...
    comment:                   Surface product; mesoscale eddies
    input_product_reference:   https://www.aviso.altimetry.fr/en/data/product...
    date_created:              2017-09-22T10:20:17Z
    time_coverage_duration:    P8771D
    time_coverage_start:       1993-01-01T00:00:00Z
    time_coverage_end:         2017-01-06T00:00:00Z
    standard_name_vocabulary:  NetCDF Climate and Forecast (CF) Metadata Conv...
    product_version:           1.0

In [4]:
print('bird',ds_bird.lon.min().data,ds_bird.lon.max().data)
for name in data:
    ds = data[name]
    print(name,ds.lon.min().data,ds.lon.max().data)

bird 170.7259979248047 234.58999633789062
aviso 0.125 359.875
wnd 0.125 359.875
sst 0.0 359.79998779296875
color 0.02083333395421505 359.9791564941406
topo 0.016666666666664165 360.0


# Collocate all data with eddy

In [5]:
%%time
#ds_data=data['aviso']
#tem = ds_data.interp(time=ds_bird.time64,lat=ds_bird.lat,lon=ds_bird.lon,assume_sorted=False)
#tem.load()

Wall time: 0 ns


In [6]:
#data,clim = get_data()
ilen_bird1 = len(ds_bird.track)
ilen_bird2 = len(ds_bird.observation_number)

for name in data:
    ds_bird = ds_bird_save.copy(deep=True)
    ds_data=data[name]
    print('data',name)
    if name=='color': 
        continue
    for var in ds_data:
        var_tem=var
        ds_bird[var_tem]=xr.DataArray(np.nan*np.empty((ilen_bird1,ilen_bird2), 
                                                      dtype=str(ds_data[var].dtype)), 
                                      coords={'track': ds_bird.track,'observation_number':ds_bird.observation_number},
                                      dims=('track','observation_number'))
        ds_bird[var_tem].attrs=ds_data[var].attrs
    if name=='topo':
        dtvar = np.timedelta64(9,'h') if name=='mur_sst' else np.timedelta64(0,'h')   # if MUR SST need to change to 9am
        tem = ds_data.interp(lat=ds_bird.lat,lon=ds_bird.lon,assume_sorted=False)
        tem = tem.load()
        for var in ds_data:
            var_tem=var
            ds_bird[var_tem][:,:]=tem[var].data
    else:
        dtvar = np.timedelta64(9,'h') if name=='mur_sst' else np.timedelta64(0,'h')   # if MUR SST need to change to 9am
        tem = ds_data.interp(time=ds_bird.time64+dtvar,lat=ds_bird.lat,lon=ds_bird.lon,assume_sorted=False)
        tem = tem.load()
        for var in ds_data:
            var_tem=var
            ds_bird[var_tem][:,:]=tem[var].data
    #output data
    ds_bird.to_netcdf(filename_origin_out+name+'.nc')
    print('output',filename_origin_out+name+'.nc')

data aviso
output f:/data/project_data/NASA_biophysical/collocated_data/NPPSD_GOA_allseabird_full_eddy_info_envdataaviso.nc
data wnd
output f:/data/project_data/NASA_biophysical/collocated_data/NPPSD_GOA_allseabird_full_eddy_info_envdatawnd.nc
data sst
output f:/data/project_data/NASA_biophysical/collocated_data/NPPSD_GOA_allseabird_full_eddy_info_envdatasst.nc
data color
data topo
output f:/data/project_data/NASA_biophysical/collocated_data/NPPSD_GOA_allseabird_full_eddy_info_envdatatopo.nc


In [ ]:
ds_bird = ds_bird_save.copy(deep=True)


In [ ]:
#********************************************************
#read in old data
ds_bird = ds.open_dataset(filename_origin_out)
ds_bird.close()
ds_bird_save = ds_bird.copy(deep=True)


In [ ]:
#color
ilen_bird1 = len(ds_bird.track)
ilen_bird2 = len(ds_bird.observation_number)

for name in data:
    ds_data=data[name]
    print('data',name)
    if (name=='aviso') or (name=='wnd') or (name=='sst') or (name=='topo'): 
        continue
    for var in ds_data:
        var_tem=var
        ds_bird[var_tem]=xr.DataArray(np.nan*np.empty((ilen_bird1,ilen_bird2), 
                                                      dtype=str(ds_data[var].dtype)), 
                                      coords={'track': ds_bird.track,'observation_number':ds_bird.observation_number},
                                      dims=('track','observation_number'))
        ds_bird[var_tem].attrs=ds_data[var].attrs
    if name=='topo':
        dtvar = np.timedelta64(9,'h') if name=='sst' else np.timedelta64(0,'h')   # if MUR SST need to change to 9am
        tem = ds_data.interp(lat=ds_bird.lat,lon=ds_bird.lon,assume_sorted=False)
        tem = tem.load()
        for var in ds_data:
            var_tem=var
            ds_bird[var_tem][:,:]=tem[var].data
    else:
        for i in range(ilen_bird1):
            dtvar = np.timedelta64(9,'h') if name=='sst' else np.timedelta64(0,'h')   # if MUR SST need to change to 9am
            try:
                tem = ds_data.interp(time=ds_bird.time64[i,:]+dtvar,lat=ds_bird.lat[i,:],lon=ds_bird.lon[i,:],assume_sorted=False)
                tem = tem.load()
            except:
                print('exception',i,name)
                pass
            for var in ds_data:
                var_tem=var
                ds_bird[var_tem][i,:]=tem[var].data
            if int(i/50)*50==i:
                #output data
                ds_bird.to_netcdf(filename_origin_out+name+'.nc')
                print('output',i,filename_origin_out+name+'.nc')
    #output data
    ds_bird.to_netcdf(filename_origin_out+name+'.nc')
    print('output',filename_origin_out+name+'.nc')

data aviso
data wnd
data sst
data color
output 0 f:/data/project_data/NASA_biophysical/collocated_data/NPPSD_GOA_allseabird_full_eddy_info_envdatacolor.nc
output 50 f:/data/project_data/NASA_biophysical/collocated_data/NPPSD_GOA_allseabird_full_eddy_info_envdatacolor.nc
output 100 f:/data/project_data/NASA_biophysical/collocated_data/NPPSD_GOA_allseabird_full_eddy_info_envdatacolor.nc
output 150 f:/data/project_data/NASA_biophysical/collocated_data/NPPSD_GOA_allseabird_full_eddy_info_envdatacolor.nc
output 200 f:/data/project_data/NASA_biophysical/collocated_data/NPPSD_GOA_allseabird_full_eddy_info_envdatacolor.nc
output 250 f:/data/project_data/NASA_biophysical/collocated_data/NPPSD_GOA_allseabird_full_eddy_info_envdatacolor.nc
output 300 f:/data/project_data/NASA_biophysical/collocated_data/NPPSD_GOA_allseabird_full_eddy_info_envdatacolor.nc
output 350 f:/data/project_data/NASA_biophysical/collocated_data/NPPSD_GOA_allseabird_full_eddy_info_envdatacolor.nc
exception color
output 400 

In [ ]:
adir_data = 'F:/data/project_data/NASA_biophysical/collocated_data/CPR/'
filename_origin_in=adir_data+'All_CPR_Sample_catalogue_with_eddy_info_2020_10_06_origin_file.nc'
filename_origin_out=adir_data+'All_CPR_Sample_catalogue_with_eddy_info_2020_10_06_origin_file_with_data'
#read in old data
ds_bird = ds.open_dataset(filename_origin_out)
ds_bird.close()
ds_bird_save = ds_bird.copy(deep=True)
#color
ilen_bird1 = len(ds_bird.track)
ilen_bird2 = len(ds_bird.observation_number)

for name in data:
    ds_data=data[name]
    print('data',name)
    if (name=='aviso') or (name=='wnd') or (name=='sst') or (name=='topo'): 
        continue
    for var in ds_data:
        var_tem=var
        ds_bird[var_tem]=xr.DataArray(np.nan*np.empty((ilen_bird1,ilen_bird2), 
                                                      dtype=str(ds_data[var].dtype)), 
                                      coords={'track': ds_bird.track,'observation_number':ds_bird.observation_number},
                                      dims=('track','observation_number'))
        ds_bird[var_tem].attrs=ds_data[var].attrs
    if name=='topo':
        dtvar = np.timedelta64(9,'h') if name=='sst' else np.timedelta64(0,'h')   # if MUR SST need to change to 9am
        tem = ds_data.interp(lat=ds_bird.lat,lon=ds_bird.lon,assume_sorted=False)
        tem = tem.load()
        for var in ds_data:
            var_tem=var
            ds_bird[var_tem][:,:]=tem[var].data
    else:
        for i in range(700,ilen_bird1):
            dtvar = np.timedelta64(9,'h') if name=='sst' else np.timedelta64(0,'h')   # if MUR SST need to change to 9am
            try:
                tem = ds_data.interp(time=ds_bird.time64[i,:]+dtvar,lat=ds_bird.lat[i,:],lon=ds_bird.lon[i,:],assume_sorted=False)
                tem = tem.load()
            except:
                print('exception',i,name)
                pass
            for var in ds_data:
                var_tem=var
                ds_bird[var_tem][i,:]=tem[var].data
            if int(i/50)*50==i:
                #output data
                ds_bird.to_netcdf(filename_origin_out+name+'.nc')
                print('output',i,filename_origin_out+name+'.nc')
    #output data
    ds_bird.to_netcdf(filename_origin_out+name+'.nc')
    print('output',filename_origin_out+name+'.nc')

In [ ]:
# check for exceptions getting ocean color data
iarg = np.argwhere(np.isnan(ds_bird.chlor_a[:,0]))
iarg

In [ ]:
i

In [ ]:
for i in range(690,ilen_bird1):
    dtvar = np.timedelta64(9,'h') if name=='sst' else np.timedelta64(0,'h')   # if MUR SST need to change to 9am
    try:
        tem = ds_data.interp(time=ds_bird.time64[i,:]+dtvar,lat=ds_bird.lat[i,:],lon=ds_bird.lon[i,:],assume_sorted=False)
        tem = tem.load()
    except:
        print('exception at ',i)
        pass # doing nothing on exception
    for var in ds_data:
        var_tem=var
        ds_bird[var_tem][i,:]=tem[var].data
    if int(i/50)*50==i:
        #output data
        ds_bird.to_netcdf(filename_origin_out+name+'.nc')
        print('output',i,filename_origin_out+name+'.nc')


# put all the files together

In [ ]:
import glob 
#put it all together and create a csv file
#print(filename_origin_out)
#filename = glob.glob('f:/data/project_data/NASA_biophysical/collocated_data/'+filename_origin_out+'*.nc')
filename = glob.glob(filename_origin_out+'*.nc')
#filename = filename[1:]
filename


In [ ]:
print(filename[0])
ds = xr.open_dataset(filename[0])
for iname in range(1,len(filename)):
    print(filename[iname])
    ds2 = xr.open_dataset(filename[iname])
    for var in ds2:
        if not var in ds:
            print(var)
            ds[var]=ds2[var]               
ds.to_netcdf(filename_origin_out+'all'+'.nc')
df_bird = ds.to_dataframe()
df_bird.to_csv(filename_origin_out+'all'+'.csv')


In [ ]:
for i in range(100):
    if int(i/10)*10==i:
        print(i)

In [ ]:
plt.scatter(ds.lon,ds.lat,c=ds.sla)

In [ ]:
data['color']

# output data

In [ ]:
filename_out=adir_data + 'All CPR Sample catalogue with eddy info_version2020_10_07.nc'
ds_eddy.to_netcdf(filename_out)

In [ ]:
ds_bird.to_netcdf(filename_origin_out+name+'.nc')

# NOW, the point of this is to look up collocated eddy information and get the history of the data.  Steps are:
1. Read in list of collocated eddies.
2. Create list of unique eddy ID
3. Read in full eddy database and select eddy id
4. collocate environmental data for entire eddy history
5. save file

In [ ]:
#ds_all = get_all_eddy(filename_aviso_all)

In [ ]:
ds_eddy = get_eddy(filename_cpr_all)
ds_eddy

In [ ]:
#drop all data where eddy radius < distance to eddy
#find unique id & create a list
subset = ds_eddy.where(ds_eddy.cpr_eddy_data_radius-ds_eddy.cpr_eddy_data_distance>0,drop=True)
_, index = np.unique(subset['cpr_eddy_data_track'], return_index=True)
eddy_list = subset['cpr_eddy_data_track'][index]
print(eddy_list[0])
#(subset.cpr_eddy_data_radius-subset.cpr_eddy_data_distance).plot()

In [ ]:
data,clim = get_data()

In [ ]:
eddy_list.size

In [ ]:
eddy_list[0]

In [ ]:
for id in eddy_list:
    if id in [118997,133297,149182,150094,181103,223608,233920,237425,241096]:
        print(id.data)

In [ ]:




#for ieddy in eddy_list:
for ieddy in range(eddy_list.size):
    if ieddy<8:
        continue
    subset = ds_all.where(ds_all.track==eddy_list[ieddy],drop=True)
    tt=np.empty(subset.obs.size,dtype='datetime64[ns]') 
    for i in range(subset.obs.size):
        tstr=str(subset.time[i].dt.year.data)+'-'+str(subset.time[i].dt.month.data).zfill(2)+'-'+str(subset.time[i].dt.day.data).zfill(2)
        tem=np.datetime64(tstr)
        tt[i]=tem
    subset['time']=xr.DataArray(tt,dims=['obs'])  
    for name in data:
        ds_data=data[name]
        if name=='topo':
            continue
        print('name',name)
        for var in ds_data:
            var_tem=ds_data[var].attrs['var_name']
            subset[var_tem]=subset.latitude.copy(deep=True)*np.NaN
            subset[var_tem].attrs=ds_data[var].attrs
        print('var',var_tem)
        for i in range(subset.latitude.size):
            lat1,lat2=subset.latitude[i].data-1,subset.latitude[i].data+1
            lon1,lon2=subset.longitude[i].data-1,subset.longitude[i].data+1
            #interp in time and select region around lat/lon to subset before loading data
            #interp doesn't work on chunked dims so rechunk
            ds_data2 = ds_data.chunk({'time':ds_data.time.size,'lat':ds_data[var].chunks[1],'lon':ds_data[var].chunks[2]})
            #ds_data2.interp(time=subset.time[i].data)
            tem = ds_data2.interp(time=subset.time[i].data).sel(lat=slice(lat1,lat2),lon=slice(lon1,lon2)).load()
            tem = tem.interp(lat=subset.latitude[i].data,lon=subset.longitude[i].data)
            for var in ds_data:
                var_tem=ds_data[var].attrs['var_name']
                subset[var_tem][i]=tem[var]
    for name in clim:
        ds_data=clim[name]
        print('name',name)
        for var in ds_data:
            var_tem=ds_data[var].attrs['var_name']+'_clim'
            subset[var_tem]=subset.latitude.copy(deep=True)*np.NaN
            subset[var_tem].attrs=ds_data[var].attrs
        print('var',var_tem)
        for i in range(subset.latitude.size):
            lat1,lat2=subset.latitude[i].data-1,subset.latitude[i].data+1
            lon1,lon2=subset.longitude[i].data-1,subset.longitude[i].data+1
            #interp in time and select region around lat/lon to subset before loading data
            ds_data2 = ds_data.chunk({'dayofyear':ds_data.time.size,'lat':ds_data[var].chunks[1],'lon':ds_data[var].chunks[2]})
            #ds_data2.interp(time=subset.time[i].data)
            tem = ds_data2.sel(dayofyear=subset.time[i].dt.dayofyear.data).sel(lat=slice(lat1,lat2),lon=slice(lon1,lon2)).load()
            tem = tem.interp(lat=subset.latitude[i].data,lon=subset.longitude[i].data)
            for var in ds_data:
                var_tem=ds_data[var].attrs['var_name']+'_clim'
                subset[var_tem][i]=tem[var]
    ds_topo=data['topo']
    subset['ETOPO_depth']=ds_topo.z.interp(lat=subset.latitude,lon=subset.longitude,method='nearest')   
    filename_out='F:/data/NASA_biophysical//collocated_data/eddy_collocated_data'+str(ieddy).zfill(8)+'.nc'
    subset.to_netcdf(filename_out)    

In [ ]:
ds_data

In [ ]:
            ds_data2 = ds_data.chunk({'time':ds_data.time.size,'lat':ds_data[var].chunks[1],'lon':ds_data[var].chunks[2]})


In [ ]:
ds_data[var].chunks

In [ ]:
ds_data[var]

In [ ]:
def get_eddy(filename):
    ds_eddy = xr.open_dataset(filename)
    tt=np.empty(ds_eddy.etopo_depth.size,dtype='datetime64[ns]') 
    for i in range(ds_eddy.etopo_depth.size):
        tstr=str(ds_eddy.cpr_sample_year[i].data)+'-'+str(ds_eddy.cpr_sample_month[i].data).zfill(2)+'-'+str(ds_eddy.cpr_sample_day[i].data).zfill(2)
        tem=np.datetime64(tstr)
        tt[i]=tem
    ds_eddy['cpr_sample_time']=xr.DataArray(tt,dims=['z'])
    return ds_eddy

def get_all_eddy(filename):
    ds = xr.open_dataset(filename)
    ds['longitude'] = (ds['longitude'] + 180) % 360 - 180
    ds_eddy = ds
#    tt=np.empty(ds_eddy.obs.size,dtype='datetime64[ns]') 
#    for i in range(ds_eddy.obs.size):
#        tstr=str(ds_eddy.time[i].dt.year.data)+'-'+str(ds_eddy.time[i].dt.month.data).zfill(2)+'-'+str(ds_eddy.time[i].dt.day.data).zfill(2)
#        tem=np.datetime64(tstr)
#        tt[i]=tem
#    ds_eddy['cpr_sample_time']=xr.DataArray(tt,dims=['obs'])
    return ds_eddy


In [ ]:
fig, (ax1) = plt.subplots(nrows=1, figsize=(6, 5.4))
im = ax1.imshow(ds_topo.z[7000:9500,0:4500], interpolation='bilinear',vmin=-7000.0, vmax=1.0,aspect='auto',origin='lower')
plt.show()